In this code, we aim to use Edge Pooling and Unpooling in order to reconstruct the events. The main message passing technique so far is SAGE, and the main loss functions are MSE and Chamfer Distance 

# Libraries Installation


Import Pytorch Version 1.4 for compatibility with some libraries to be used like neural-net pytorch that contains EMD and Chamfer Distance Libraries

In [1]:
#!pip install torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html    #!pip install -q torch==1.4.0 torchvision==0.5

In [2]:
"""
!pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-cluster==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-geometric
"""

'\n!pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html\n!pip install torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html\n!pip install torch-cluster==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html\n!pip install torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html\n!pip install torch-geometric\n'

In [3]:
#pip install "neuralnet-pytorch[geom] @ git+git://github.com/justanhduc/neuralnet-pytorch.git@master"

In [4]:
#!pip install neuralnet-pytorch

In [5]:
#!pip install git+git://github.com/justanhduc/neuralnet-pytorch.git@fancy

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
cd drive/My\ Drive/Gitlink/Version2_Pooling

/content/drive/My Drive/Gitlink/Version2_Pooling


In [8]:
import argparse
import time
import torch_geometric
import numpy as np
import scipy.sparse as sp
import torch
from torch import optim
from torch_geometric.data import Data

from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F
from torch.utils.data import Dataset, IterableDataset, DataLoader
from itertools import cycle, islice
from torch_geometric.utils import dropout_adj


In [9]:

from optimizer import loss_function
import iterableJet
from iterableJet import IterableMuons
import class_jet
from class_jet import FCMuonsGPU
from model import GraphPooling
import model_trial

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")


parser = argparse.ArgumentParser()
parser.add_argument('--seed', type=int, default=42, help='Random seed.')
parser.add_argument('--epochs', type=int, default=50, help='Number of epochs to train.')
parser.add_argument('--batch_size', type=int, default=25, help='Initial learning rate.') #100

parser.add_argument('--lr', type=float, default=0.001, help='Initial learning rate.') #0.001
parser.add_argument('--dropout', type=float, default=0.4, help='Dropout rate (1 - keep probability).')
args = parser.parse_args([])
torch.backends.cudnn.benchmark = True

In [10]:
import DiffAE

In [11]:
samples=torch.load('../../Unet/ECAL_f01_june.pt')

In [12]:
#samples=samples[:30000]

In [13]:
"""
for sam in samples:
  sam['x'][:,2]=-torch.log(sam['x'][:,2])*15
  #sam['x'][:,:2]/=150
"""

"\nfor sam in samples:\n  sam['x'][:,2]=-torch.log(sam['x'][:,2])*15\n  #sam['x'][:,:2]/=150\n"

In [14]:
model3=DiffAE.GraphAE()
model3.train()
model3.cuda()

GraphAE(
  (enc): Encoder(
    (gnn1_pool): GNN(
      (conv1): DenseSAGEConv(3, 64)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): DenseSAGEConv(64, 64)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): DenseSAGEConv(64, 1224)
      (bn3): BatchNorm1d(1224, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (lin): Linear(in_features=1352, out_features=1224, bias=True)
    )
    (gnn1_embed): GNN(
      (conv1): DenseSAGEConv(3, 64)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): DenseSAGEConv(64, 64)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): DenseSAGEConv(64, 64)
      (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (gnn2_pool): GNN(
      (conv1): DenseSAGEConv(192, 64)
      

In [15]:
#model2=model_trial.Net()
#model2.train()
#model2.cuda()

In [16]:
"""
model = GraphPooling(in_channels=3, out_channels1=64, out_channels2=128, out_channels3=256,out_channels4=512,out_channels5=1024,dropout=args.dropout,
                    batch_size=args.batch_size)
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=0.001)
#optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001, weight_decay=0.001)
"""

'\nmodel = GraphPooling(in_channels=3, out_channels1=64, out_channels2=128, out_channels3=256,out_channels4=512,out_channels5=1024,dropout=args.dropout,\n                    batch_size=args.batch_size)\noptimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=0.001)\n#optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001, weight_decay=0.001)\n'

In [17]:
optimizer = torch.optim.Adam(model3.parameters(), lr=args.lr, weight_decay=0.001)

In [18]:
def sizer(samples):
    inds=[]
    for k in samples:
        inds.append(k['x'].shape[0])
    return inds

sizes=sizer(samples[:58000])

In [19]:
maxCount=np.max(np.array(sizes))
"""
refMat=np.zeros((args.batch_size,maxCount))
for pi in range(refMat.shape[0]):
  refMat[pi,:sizes[pi]]=1
"""

'\nrefMat=np.zeros((args.batch_size,maxCount))\nfor pi in range(refMat.shape[0]):\n  refMat[pi,:sizes[pi]]=1\n'

In [20]:
def ref(bsize,maxC,i1,i2):
  maxCount=np.max(np.array(sizes)) ##max num of nodes
  refMat=np.zeros((bsize,maxC)) ## matrix of zeros
  for pi in range(i1,i2):##10
    refMat[10-(i2-pi),:sizes[pi]]=1 ## fill ones 
  return refMat

In [21]:
fin=[]
countit=0
for m in sizes:
    fin.append(np.repeat(countit,m))
    countit+=1


In [22]:

from torch_geometric.data import DataLoader
#jet=IterableMuons(samples)
loader = DataLoader(samples[:58000], batch_size=args.batch_size,num_workers=2,pin_memory=True)

#%%
#model.train()
#model= model.to(device)
#optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=0.001)
#optimizer = torch.optim.RMSprop(model.parameters(), lr=0.0005, weight_decay=0.001)
#model.train()
#model.to(device)


In [23]:
scheduler=StepLR(optimizer, step_size=4, gamma=0.1)

In [24]:
testing = samples[-1000:]

sizes2=sizer(testing)


In [25]:

fin2=[]
count23=0
for m2 in sizes2:
    fin2.append(np.repeat(count23,m2))
    count23+=1

from torch_geometric.data import DataLoader
Testloader = DataLoader(testing, batch_size=args.batch_size)

In [26]:
from torch_geometric.utils import to_dense_batch, to_dense_adj

In [27]:
def to_sparse_batch(x, adj, mask=None):
    # transform x (B x N x D), adj (B x N x N), mask (B x N), here N is N_max
    # to x, edge_index, edge_attr/weight, batch

    B, N_max, D = x.shape
    # get num of nodes and reshape x
    num_nodes_graphs = torch.zeros_like(x[:,0,0], dtype=torch.int64).fill_(N_max)
    x = x.reshape(-1, D) # total_nodes * D

    # apply mask 
    if mask is not None:
        # mask adj
        adj = (adj * mask.unsqueeze(2)).transpose(1,2) 
        adj = (adj * mask.unsqueeze(2)).transpose(1,2) 
        # get number nodes per graph 
        num_nodes_graphs = mask.sum(dim=1)  # B
        # mask x
        x = x[mask.reshape(-1)] # total_nodes * D

    # get weight and index
    edge_weight = adj[adj.nonzero(as_tuple=True)]
    nnz_index = adj.nonzero().t()
    graph_idx, edge_index = nnz_index[0], nnz_index[1:]

    # get offset with size B
    offset_graphs = torch.cumsum(num_nodes_graphs, dim=0) # B
    offset_graphs = torch.cat([offset_graphs.new_zeros(1), offset_graphs]) # B+1
    
    # update edge index by adding offset
    edge_index += offset_graphs[graph_idx]

    # set up batch
    batch = torch.repeat_interleave(input=torch.arange(B, device=x.device), repeats=num_nodes_graphs )

    return x, edge_index, edge_weight, batch

In [30]:

for epoch in range(50):
    #model.train()
    
    count=0
    c1,c2=0,args.batch_size
    epLoss=0
    t = time.time()
    for el in islice(loader,0,2320):#count2,el in enumerate(loader):
        
        gra=el.x
        adj=el.edge_index

        lengs=torch.LongTensor(np.hstack(np.array(fin[c1:c2])-c1))
        count+=1
        refMat=ref(args.batch_size,maxCount,c1,c2)
        #whole=to_dense_batch(el.x, lengs, fill_value=0, max_num_nodes=refMat.shape[1])
        #wholeAdj=to_dense_adj(adj, lengs, edge_attr=None, max_num_nodes=refMat.shape[1])

        optimizer.zero_grad()

        r1 ,adj1,mu,sig= model3(gra.cuda(),adj.cuda(),lengs.cuda(),torch.from_numpy(refMat).cuda())

        loss = loss_function(r1,gra,lengs,refMat,mu,sig)    

        loss.backward()

        optimizer.step()

        cur_loss = loss.item()
        
        epLoss+=float(cur_loss)

        c1+=args.batch_size
        c2+=args.batch_size
        
        if count%1160==0:
            print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(epLoss/count),"time=", "{:.5f}".format(time.time() - t))
            t = time.time()

            with torch.no_grad():
              model3.eval()
              VaLoss=0
              d1,d2=0,args.batch_size
              for el2 in islice(Testloader,40):
                gra2=el2.x.cuda()  ##features 
                adj2=el2.edge_index.cuda() ## edge matrix 
                lengs2=torch.LongTensor(np.hstack(np.array(fin2[d1:d2])-d1)).cuda()
                refMat2=ref(args.batch_size,maxCount,d1,d2)

                r12,mu2,sig2= model3(gra2,adj2,lengs2,torch.from_numpy(refMat2).cuda()) ## X , A 
                loss2 = loss_function(r12,gra2,lengs2,refMat,mu2,sig2)
                cur_loss2 = loss2.item()
                VaLoss+=float(cur_loss2)
                d1+=args.batch_size
                d2+=args.batch_size
              print("Val Loss:", '%04d' % (VaLoss/10))

    if epoch%2==0:
      scheduler.step()
      torch.save({
            'epoch': epoch,
            'model_state_dict': model3.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'epoch':epoch,
            'loss': loss,
            'epLoss':epLoss
            }, './diffPool.pth')

           

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


KeyboardInterrupt: ignored

In [31]:
count

5

In [ ]:
lengs.numpy()-(c1)

In [ ]:
tets=to_dense_batch(gra, lengs, fill_value=0, max_num_nodes=refMat.shape[1])

In [32]:
loss

tensor(12904.0605, device='cuda:0', grad_fn=<AddBackward0>)

In [ ]:

#%%
testing = samples[-330:]

#%%
sizes2=sizer(testing)


In [ ]:
            
posMom=[]

            with torch.no_grad():
              model.eval()
              for el2 in islice(Testloader,1):
                gra2=el2.x.cuda()  ##features 
                adj2=el2.edge_index.cuda() ## edge matrix 
                lengs2=torch.LongTensor(np.hstack(np.array(fin2[:100]))).cuda()
                r12,mu2,sig2= model(gra2,adj2,lengs2,gra2.shape[0]) ## X , A 
                loss2 = loss_function(r12,gra2,lengs2,mu2,sig2)
                cur_loss2 = loss2.item()
                posMom.append(r12)
                print("Val Loss:", '%04d' % (cur_loss2))

In [ ]:

#%%
alles=[]
temp=0
for siz in sizes2:
  temp+=siz
  alles.append(temp)

In [ ]:
 %matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib.colors import LogNorm

In [ ]:
real=testing[1]['x']

In [ ]:
%matplotlib inline
#xmin, xmax = min(lis4[:,0]), max(lis4[:,0])
#ymin, ymax = min(lis4[:,1]), max(lis4[:,1])
for k in range(10):
  real=testing[k]['x']
  xmin,xmax=0,125
  ymin,ymax=0,125
  binsxy = [int((xmax - xmin) / 50), int((ymax - ymin) / 50)]
  weights=np.exp(-real[:,2]/10)
  plt.figure(figsize=(7,6))
  sc = plt.scatter(real[:,0],real[:,1],c=weights,cmap='viridis',norm=LogNorm(), alpha=0.9)
  plt.colorbar(sc)
  check=np.arange(0,140,10)
  plt.xticks(check)
  plt.yticks(check)
  plt.grid()
  #plt.savefig('./Real2/fig'+str(k)+'.png')


In [ ]:
%matplotlib inline
#xmin, xmax = min(lis4[:,0]), max(lis4[:,0])
#ymin, ymax = min(lis4[:,1]), max(lis4[:,1])
k1,k2=0,1
for j in range(10):
  
  lis4=r12[alles[k1]:alles[k2]].detach().cpu().clone()
  xmin,xmax=0,125
  ymin,ymax=0,125
  binsxy = [int((xmax - xmin) / 50), int((ymax - ymin) / 50)]
  weights=np.exp(-lis4[:,2]/10)
  plt.figure(figsize=(7,6))
  sc = plt.scatter(lis4[:,0],lis4[:,1],c=weights,cmap='viridis', norm=LogNorm(),alpha=0.9)
  plt.colorbar(sc)
  check=np.arange(0,140,10)
  plt.xticks(check)
  plt.yticks(check)
  plt.grid()
  k1+=1
  k2+=1
  #plt.savefig('./Rec2/rec'+str(j)+'.png')


In [ ]:
  lis4=r12[215:691].detach().cpu().clone()
  xmin,xmax=0,125
  ymin,ymax=0,125
  binsxy = [int((xmax - xmin) / 50), int((ymax - ymin) / 50)]
  weights=np.exp(-10*lis4[:,2])
  plt.figure(figsize=(7,6))
  sc = plt.scatter(lis4[:,0]*100,lis4[:,1]*100,c=weights,cmap='viridis', norm=LogNorm(),alpha=0.9)
  plt.colorbar(sc)
  check=np.arange(0,140,10)
  plt.xticks(check)
  plt.yticks(check)
  plt.grid()

In [ ]:
%matplotlib inline
#xmin, xmax = min(lis4[:,0]), max(lis4[:,0])
#ymin, ymax = min(lis4[:,1]), max(lis4[:,1])

k1,k2=0,1
for j in range(9):
  lis4=r12[alles[k1]:alles[k2]].detach().cpu().clone()
  xmin,xmax=0,125
  ymin,ymax=0,125
  binsxy = [int((xmax - xmin) / 50), int((ymax - ymin) / 50)]
  weights=np.exp(-10*lis4[:,2])
  plt.figure(figsize=(7,6))
  sc = plt.scatter(lis4[:,0]*100,lis4[:,1]*100,c=weights,cmap='viridis', norm=LogNorm(),alpha=0.9)
  plt.colorbar(sc)
  check=np.arange(0,140,10)
  plt.xticks(check)
  plt.yticks(check)
  plt.grid()
  k1+=1
  k2+=1
  
  #plt.savefig('./Rec/rec'+str(j)+'.png')
